# How to optimize with Torch
Kai Puolamäki, 28 June 2022

## Introduction

This brief tutorial demonstrates how [PyTorch](https://pytorch.org) can be used to find minimum values of arbitrary functions, as is done in [SLISEMAP](https://github.com/edahelsinki/slisemap). The advantages of PyTorch include the use of autograd and optionally GPU acceleration. These may result in significant speedups when optimizing high-dimensional loss functions, which often happens in deep learning but also elsewhere.

The existing documentation of PyTorch is geared towards deep learning. It is currently difficult to find documentation of how to do "simple" optimization without any deep learning context, which is why I wrote this tutorial in the hope that it will be useful for someone.

## Toy example

Here we minimise a simple regularized least squares loss given by
$$
L = \lVert {\bf y}-{\bf X}{\bf b} \rVert_2^2+\lVert{\bf{b}}\rVert_2^2/10,
$$
where ${\bf X}\in{\mathbb{R}}^{3\times 2}$ and ${\bf y}\in{\mathbb{R}}^3$ are constants and ${\bf{b}}\in{\mathbb{R}}^2$ is a vector whose values are to be found by the optimiser. We could optimize any reasonably behaving function; here we picked the least squares loss for simplicity.

In this example, we use the following values for the constant matrix and vector:
$$
{\bf{X}}=
\begin{pmatrix}
1 & 1 \\
1 & 2 \\
1 & 3.14159
\end{pmatrix},~~~~
{\bf{y}}=
\begin{pmatrix}
1 \\ 2 \\ 3 
\end{pmatrix}.
$$
In this example, the loss $L$ obtains a minimal value of $L=0.0887$ when ${\bf{b}}=\left(0.141,0.906\right)^T$.

## Numpy and Scipy

We first solve the problem with the [standard `scipy` optimizer](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html), by using an arbitrarily chosen initial starting point.

We first define the matrices and vectors as Numpy arrays and then define a loss function `loss_fn0` that that takes the value of ${\bf{b}}$ as input and outputs the value of the loss $L$.

In [1]:
import numpy as np
from scipy.optimize import minimize

<IPython.core.display.Javascript object>

In [2]:
X0 = np.array([[1, 1], [1, 2], [1, 3.14159]], dtype=float)
y0 = np.array([1, 2, 3], dtype=float)
b0 = np.array([0.12, 0.34], dtype=float)


def loss_fn0(b):
    return ((y0 - X0 @ b) ** 2).sum() + (b**2).sum() / 10.0


loss_fn0(b0)

5.027434485368359

<IPython.core.display.Javascript object>

For this starting point the loss value is $L=5.027$, which is clearly larger than the optimal value.

We can find the value of ${\bf{b}}$ that minimizes the loss $L$ by using a library optimization algorithm, BFGS in this case. We find the correct value of ${\bf{b}}$ and the corresponding loss:

In [3]:
res0 = minimize(loss_fn0, b0, method="BFGS")
res0.x, loss_fn0(res0.x)

(array([0.14119498, 0.90567679]), 0.08865057718228184)

<IPython.core.display.Javascript object>

It is always good to check if we have converged:

In [4]:
res0.message

'Optimization terminated successfully.'

<IPython.core.display.Javascript object>

## PyTorch

We'll repeat the same with Pytorch. First we define a helper function `LBFGS` that takes in the loss function and the variables to be optimized as input and that as a side effect updates the variables to their values at the minimum of the loss function.

The helper function uses the [Torch LBFGS optimizer](https://pytorch.org/docs/stable/generated/torch.optim.LBFGS.html). The `closure` is a function that essentially evaluates the loss function and updates the gradient values. 

You can use this helper function as a generic optimizer, much in the same way as you would use the `scipy.optimize.minimize` above by just cutting-and-pasting the LBGGS helper function into your code. The file [utils.py](https://github.com/edahelsinki/slisemap/blob/main/slisemap/utils.py) in the SLISEMAP source code contains a more advanced version of the helper function.

In [5]:
import torch


def LBFGS(loss_fn, variables, max_iter=500, line_search_fn="strong_wolfe", **kwargs):
    """Optimise a function using LBFGS.
    Args:
        loss_fn (Callable[[], torch.Tensor]): Function that returns a value to be minimised.
        variables (List[torch.Tensor]): List of variables to optimise (must have `requires_grad=True`).
        max_iter (int, optional): Maximum number of LBFGS iterations. Defaults to 500.
        line_search_fn (Optional[str], optional): Line search method (None or "strong_wolfe"). Defaults to "strong_wolfe".
        **kwargs (optional): Argumemts passed to `torch.optim.LBFGS`.
    Returns:
        torch.optim.LBFGS: The LBFGS optimiser.
    """

    optimiser = torch.optim.LBFGS(
        variables, max_iter=max_iter, line_search_fn=line_search_fn, **kwargs
    )

    def closure():
        optimiser.zero_grad()
        loss = loss_fn()
        loss.backward()
        return loss

    optimiser.step(closure)

    return optimiser

<IPython.core.display.Javascript object>

Torch functions typically require that we define the variables torch tensors. The torch tensors correspond to Numpy arrays, but they carry autograd information and they can optionally be used within a GPU. Notice that we need to attach the slot for the gradients to ${\bf{b}}$ tensor because we want to optimize it!

In [6]:
X = torch.tensor(X0, dtype=torch.float)
y = torch.tensor(y0, dtype=torch.float)
b = torch.tensor(b0, dtype=torch.float, requires_grad=True)
X, y, b

(tensor([[1.0000, 1.0000],
         [1.0000, 2.0000],
         [1.0000, 3.1416]]),
 tensor([1., 2., 3.]),
 tensor([0.1200, 0.3400], requires_grad=True))

<IPython.core.display.Javascript object>

The safe way to make Torch tensors Numpy arrays is to first move them to CPU, then detach any autograd part, and then make them numpy arrays:

In [7]:
X.cpu().detach().numpy()

array([[1.     , 1.     ],
       [1.     , 2.     ],
       [1.     , 3.14159]], dtype=float32)

<IPython.core.display.Javascript object>

Next, we define the loss function that takes no parameters as an input and which outputs the loss (a tensor with only one real number as a value). If you want to evaluate the value of loss for different values of ${\bf{b}}$ you must update the values in the corresponding tensor.

It is important to use only Torch arithmetic operations that support autograd. Luckily, there are enough operations to cover most needs. Instead of `sum` method in the [Tensor object](https://pytorch.org/docs/stable/tensors.html) as in the first example below we can alternatively use [torch.sum](https://pytorch.org/docs/stable/generated/torch.sum.html) (both of which supports torch tensors and autograd), but we cannot use, e.g., [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) (which does not support torch tensors and autograd).

In [8]:
## Using the sum method in the Tensor object:
def loss_fn():
    return ((y - X @ b) ** 2).sum() + (b**2).sum() / 10.0


## Alternate but equivalent way of writing the same thing by using torch.sum:
def loss_fn():
    return torch.sum((y - X @ b) ** 2) + torch.sum(b**2) / 10.0


## You cannot use, e.g., Numpy operations which do not support tensors and autograd:
def loss_fn_WRONG_DO_NOT_USE():
    return np.sum((y - X @ b) ** 2) + np.sum(b**2) / 10.0

<IPython.core.display.Javascript object>

Evaluating the loss function gives the value of the loss (as a tensor):

In [9]:
loss_fn()

tensor(5.0274, grad_fn=<AddBackward0>)

<IPython.core.display.Javascript object>

If we want to have the loss value as a real number then the correct procedure is to first move the tensor to CPU (this matters if we use GPU, otherwise it is a null operation), then detach the autograd component, and then take the only item out as a real number:

In [10]:
loss_fn().cpu().detach().item()

5.027434349060059

<IPython.core.display.Javascript object>

We use the helper function `LBFGS` defined above to do the optimization. We need to give as parameters the loss function and a list of tensors to be optimized. As a result, the value of the tensor ${\bf{b}}$ is updated to the value that minimizes the loss!

In [11]:
res = LBFGS(loss_fn, [b])
X, y, b

(tensor([[1.0000, 1.0000],
         [1.0000, 2.0000],
         [1.0000, 3.1416]]),
 tensor([1., 2., 3.]),
 tensor([0.1412, 0.9057], requires_grad=True))

<IPython.core.display.Javascript object>

The optimum value of the loss function is the same as in the first example with Numpy and standard Scipy optimization function.

In [12]:
loss_fn().cpu().detach().item()

0.08865057677030563

<IPython.core.display.Javascript object>

Again, it is good to check that the optimization converged successfully:

In [13]:
res.state_dict()["state"][0]["n_iter"]

6

<IPython.core.display.Javascript object>

The optimization took 6 iterations (cutoff being 500). Therefore, the optimization was probably terminated due to convergence to a local minumum and we should be fine. If there is no convergence you may need to increase the cutoff or study the matter further (e.g., the loss to be optimized could be badly behaving).

## Addendum: differences between "conventional" optimization and optimization in deep learning

The idea in optimization is to find parameter values that minimize the value of a given target function (loss). When the parameters of the target function are real-valued numbers then typically gradient-based optimizers are used. 

In deep learning applications the loss to be optimized is typically, e.g., classification error of the deep learning networks and the parameters are weights in the network. Below, I list some practical differences between optimization in deep learning and more traditional optimization problems.


### Stochastic gradient algorithms are more popular in deep learning

Deep learning problems are typically high dimensional (meaning there are lots of parameters). Stochastic gradient -based algorithms scale well for very high-dimensional datasets, while more conventional optimization methods may become too slow. However, for lower-dimensional problems the stochastic gradient -based algorithms may be slower to converge. LBFGS (which is not based on stochastic gradient) used above is very good conventional optimizer and me be a better default choice for conventional problems with a reasonable number of parameters to be optimised.


### In deep learning we do not want to find the minimum

In deep learning we do not usually want to find the parameter values that minimize the loss, because this may result into overfitting to the training data. Instead, gradient optimization is typically run iteratively step by step. The optimization is stopped when validation loss stops decreasing. In a normal Torch workflow `optimiser.step(closure)` would be run repeatedly until validation loss stops decreasing. 

A more traditional approach for optimization (used also by the Scipy `minimize` above) is to run the optimizer until the pre-defined stopping criteria are met, which typically means that the solution is no longer improved indicating that the optimizer has found a local minimum of the loss function; this is what our LBFGS helper function does. We need to run LBFGS optimizer only one step, which is in most cases enough to converge, because the default maximum number of iterations within step is in LBFGS function set to 500, Torch default being 20. Often, the optimizer stops before 500 steps are used after convergence, but you should check this.

### In deep learning speed is considered more important than stability or robustness

In deep learning stability or robustness of the algorithm is often considered less important than scalability. If the optimization does not converge it can be just restarted with different parameters, while in a more conventional setup you would be happy if the optimizer behaves predictably and you do not have to fiddle with parameters. Therefore line search is not by default used in Torch LBFGS optimizer. I have added line search option, which guarantees that the value of the loss function does not increase at any iteration. This results to better numerical stability with the penalty of slightly longer runtime.